In [ ]:
import json
import pandas as pd
import numpy as np
import psycopg2
import requests

def lambda_handler(event, context):
    # connect to an API
    url = 'https://data.bs.ch/api/explore/v2.1/catalog/datasets/100294/records?order_by=timestamp%20DESC&limit=1'
    response = requests.get(url)
    data = response.json()
    
    # Check if data is not empty
    if not data:
        print("No data received from the API.")
        return

    # Transform it
    df = pd.DataFrame(data['results'])
    
    # Load into database
    try: 
        conn = psycopg2.connect("host=weatherdatalake.ckapktk3je1k.us-east-1.rds.amazonaws.com dbname=weatherdatalake user=nbarnett password=cardsmlb")
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)
        return
    
    try: 
        cur = conn.cursor()
    except psycopg2.Error as e: 
        print("Error: Could not get curser to the Database")
        print(e)
        return
    
    conn.set_session(autocommit=True)
    
    
    # Insert DataFrame record (assuming 'weather' table structure matches DataFrame columns)
    try:
        for index, row in df.iterrows():
            cur.execute(
                "INSERT INTO weather (timestamp, precipitation, relativehumidityhc, solarradiation, airtemperaturehc, windspeedultrasonic, winddirultrasonic) \
                VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (
                    row['timestamp'],
                    row['precipitation'],
                    row['relativehumidityhc'],
                    row['solarradiation'],
                    row['airtemperaturehc'],
                    row['windspeedultrasonic'],
                    row['winddirultrasonic']
                )
            )
    except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        return
    
    cur.close()
    conn.close()
    